# 1-3,文本数据建模流程范例

In [ ]:
import os

#mac系统上pytorch和matplotlib在jupyter中同时跑需要更改环境变量
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [ ]:
# 使用pip安装gensim库，用于自然语言处理和主题建模等任务
!pip install gensim

# 使用pip安装torchkeras库，它是一个用于PyTorch深度学习框架的高级封装，简化了模型训练和评估的流程
!pip install torchkeras

In [ ]:
import torch
import gensim
import torchkeras

print("torch.__version__ = ", torch.__version__)
print("gensim.__version__ = ", gensim.__version__)
print("torchkeras.__version__ = ", torchkeras.__version__)


### 一，准备数据

imdb数据集的目标是根据电影评论的文本内容预测评论的情感标签。

训练集有20000条电影评论文本，测试集有5000条电影评论文本，其中正面评论和负面评论都各占一半。

文本数据预处理较为繁琐，包括文本切词，构建词典，编码转换，序列填充，构建数据管道等等。


此处使用gensim中的词典工具并自定义Dataset。

下面进行演示。


![](./data/电影评论.jpg)

In [ ]:
# 导入pandas库，用于数据处理和操作
import pandas as pd

# 定义最大文本长度，每个样本保留200个词的长度
MAX_LEN = 200

# 定义批量大小，每个批次包含20个样本
BATCH_SIZE = 20

# 使用pandas库读取训练数据集，数据以制表符分隔，没有列名，分别存储在dftrain中
dftrain = pd.read_csv("./eat_pytorch_datasets/imdb/train.tsv", sep="\t", header=None, names=["label", "text"])

# 使用pandas库读取验证数据集，数据以制表符分隔，没有列名，分别存储在dfval中
dfval = pd.read_csv("./eat_pytorch_datasets/imdb/test.tsv", sep="\t", header=None, names=["label", "text"])


In [ ]:
# 导入gensim库中的corpora模块，用于构建词典和文本处理
from gensim import corpora

# 导入string库，用于文本处理，去除标点符号
import string


# 1. 文本切词函数
def textsplit(text):
    # 创建一个标点符号转换器，用于去除文本中的标点符号
    translator = str.maketrans('', '', string.punctuation)

    # 将文本按照空格切分成词汇列表，并去除标点符号
    words = text.translate(translator).split(' ')
    return words


# 2. 构建词典
# 使用corpora库中的Dictionary类构建词典
vocab = corpora.Dictionary((textsplit(text) for text in dftrain['text']))

# 过滤掉出现频率过低或过高的词汇
vocab.filter_extremes(no_below=5, no_above=5000)

# 添加特殊的标记词汇，如'<pad>'和'<unk>'，并更新词典
special_tokens = {'<pad>': 0, '<unk>': 1}
vocab.patch_with_special_tokens(special_tokens)

# 获取词典的大小
vocab_size = len(vocab.token2id)
print('vocab_size = ', vocab_size)


# 3. 序列填充函数
def pad(seq, max_length, pad_value=0):
    n = len(seq)
    # 将序列扩展到指定的最大长度，并用pad_value填充
    result = seq + [pad_value] * max_length
    return result[:max_length]


# 4. 编码转换函数
def text_pipeline(text):
    # 将文本切分成词汇，并使用词典将词汇编码为整数序列
    tokens = vocab.doc2idx(textsplit(text))

    # 将未在词典中找到的词汇替换为'<unk>'的编码
    tokens = [x if x > 0 else special_tokens['<unk>'] for x in tokens]

    # 对编码后的序列进行填充，使其达到最大长度
    result = pad(tokens, MAX_LEN, special_tokens['<pad>'])
    return result


# 测试text_pipeline函数，将文本转换为编码序列
print(text_pipeline("this is an example!"))


In [ ]:
# 导入PyTorch中的数据集和数据加载器相关模块
from torch.utils.data import Dataset, DataLoader


# 创建自定义的ImdbDataset类，用于加载IMDb电影评论数据集
class ImdbDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        # 返回数据集的长度，即样本的数量
        return len(self.df)

    def __getitem__(self, index):
        # 根据索引获取单个样本的文本和标签
        text = self.df["text"].iloc[index]
        label = torch.tensor([self.df["label"].iloc[index]]).float()

        # 使用之前定义的text_pipeline函数将文本转换为编码序列
        tokens = torch.tensor(text_pipeline(text)).int()

        # 返回编码序列和对应的标签
        return tokens, label


# 创建训练集和验证集的数据集实例
ds_train = ImdbDataset(dftrain)  # 训练集
ds_val = ImdbDataset(dfval)  # 验证集


In [ ]:
# 创建训练集和验证集的数据加载器，用于批量加载数据
# ds_train: 训练集的数据集对象
# ds_val: 验证集的数据集对象
# batch_size: 每个批次的样本数量
# shuffle: 是否在每个epoch中打乱数据顺序

# 创建训练集的数据加载器，每个批次包含50个样本，并在每个epoch中打乱数据顺序
dl_train = DataLoader(ds_train, batch_size=50, shuffle=True)

# 创建验证集的数据加载器，每个批次包含50个样本，不打乱数据顺序（用于评估模型性能）
dl_val = DataLoader(ds_val, batch_size=50, shuffle=False)


In [ ]:
# 迭代训练集数据加载器 dl_train 中的第一个批次数据
for features, labels in dl_train:
    # features: 一个批次的特征数据，通常是文本编码序列
    # labels: 对应的标签数据，通常是与特征数据相关的目标标签
    break


### 二，定义模型

使用Pytorch通常有三种方式构建模型：使用nn.Sequential按层顺序构建模型，继承nn.Module基类构建自定义模型，继承nn.Module基类构建模型并辅助应用模型容器(nn.Sequential,nn.ModuleList,nn.ModuleDict)进行封装。

此处选择使用第三种方式进行构建。


In [ ]:
# 导入PyTorch库
import torch

# 导入PyTorch中的神经网络模块
from torch import nn

# 设置随机种子，以确保随机数生成的结果可重复
torch.manual_seed(42)


In [ ]:
# 创建自定义神经网络模型Net，继承自nn.Module
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        # 创建一个词嵌入层（Embedding Layer），用于将整数编码的词汇转换为词嵌入向量
        # num_embeddings: 词汇表的大小
        # embedding_dim: 词嵌入向量的维度
        # padding_idx: 设置padding_idx参数后，在训练过程中将填充的token始终赋值为0向量
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=3, padding_idx=0)

        # 创建一个卷积神经网络模块
        self.conv = nn.Sequential()
        self.conv.add_module("conv_1", nn.Conv1d(in_channels=3, out_channels=16, kernel_size=5))
        self.conv.add_module("pool_1", nn.MaxPool1d(kernel_size=2))
        self.conv.add_module("relu_1", nn.ReLU())
        self.conv.add_module("conv_2", nn.Conv1d(in_channels=16, out_channels=128, kernel_size=2))
        self.conv.add_module("pool_2", nn.MaxPool1d(kernel_size=2))
        self.conv.add_module("relu_2", nn.ReLU())

        # 创建一个全连接神经网络模块
        self.dense = nn.Sequential()
        self.dense.add_module("flatten", nn.Flatten())
        self.dense.add_module("linear", nn.Linear(6144, 1))  # 6144是全连接层输入维度，1是输出维度（二分类）

    def forward(self, x):
        # 输入x是文本数据的编码序列
        # 通过词嵌入层将编码序列转换为词嵌入向量，并进行维度转置
        x = self.embedding(x).transpose(1, 2)

        # 将词嵌入向量输入卷积神经网络模块中进行卷积、池化和ReLU激活操作
        x = self.conv(x)

        # 将卷积后的结果输入全连接神经网络模块中，得到最终输出y
        y = self.dense(x)

        return y


# 创建Net类的实例，即神经网络模型
net = Net()

# 打印网络结构
print(net)


卷积神经网络（Convolutional Neural Network，CNN）和全连接神经网络（Fully Connected Neural Network，FCN）是深度学习中常用的两种神经网络架构，它们包含不同类型的层，每种层都有其特定的作用。

### 卷积神经网络（CNN）：

1. **卷积层（Convolutional Layer）**：
   - 作用：用于提取输入数据中的特征。卷积操作通过滤波器（卷积核）在输入数据上滑动，执行局部感知，从输入中提取特定的特征。
   - 设置原因：卷积层通过局部连接和共享权重的方式，有效地捕捉图像、文本等数据中的空间局部关系，减少了参数数量，提高了特征提取的效率。

2. **池化层（Pooling Layer）**：
   - 作用：用于降低特征图的空间分辨率，减少计算量，并提高模型对平移不变性的学习能力。
   - 设置原因：池化层可以减小数据的尺寸，同时保留关键信息，有助于降低模型的复杂度和提高模型的泛化能力。

3. **ReLU激活层（Rectified Linear Unit Activation Layer）**：
   - 作用：引入非线性，增强模型的表达能力，使模型能够学习复杂的特征。
   - 设置原因：ReLU激活函数能够解决梯度消失问题，同时计算速度较快。

### 全连接神经网络（FCN）：

1. **全连接层（Fully Connected Layer）**：
   - 作用：每个神经元与前一层的所有神经元相连接，用于学习全局特征和实现分类、回归等任务。
   - 设置原因：全连接层可以对所有输入进行加权组合，具有较强的表达能力，适用于各种复杂任务。

2. **激活层（Activation Layer）**：
   - 作用：引入非线性，增强模型的非线性拟合能力。
   - 设置原因：激活层可以使神经网络模型具有非线性映射能力，可以学习到更复杂的函数关系。

总结起来，卷积神经网络通过卷积层、池化层和ReLU激活层构建了一个逐渐提取特征的层次结构，适用于处理具有空间局部关系的数据，如图像和文本。全连接神经网络则通过全连接层和激活层构建了一个密集连接的层次结构，适用于学习全局特征和执行各种复杂任务。这些不同层的组合和设计是为了使模型能够有效地捕捉数据的特征并实现不同的机器学习任务。

In [ ]:
# 导入torchkeras库中的summary函数
from torchkeras import summary

# 使用summary函数生成神经网络模型net的摘要信息
# net: 要生成摘要信息的神经网络模型
# input_data: 输入数据的示例，用于确定模型的输入维度
summary(net, input_data=features)

### 三，训练模型

训练Pytorch通常需要用户编写自定义训练循环，训练循环的代码风格因人而异。

有3类典型的训练循环代码风格：脚本形式训练循环，函数形式训练循环，类形式训练循环。

此处介绍一种较通用的仿照Keras风格的类形式的训练循环。

该训练循环的代码也是torchkeras库的核心代码。

torchkeras详情:  https://github.com/lyhue1991/torchkeras 



In [ ]:
# 导入所需的库和模块
import sys
import numpy as np
import datetime
from tqdm import tqdm
import torch
from torch import nn
from copy import deepcopy


# 定义一个用于打印日志的函数
def printlog(info):
    nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    print("\n" + "==========" * 8 + "%s" % nowtime)
    print(str(info) + "\n")


# 定义一个用于执行单个训练或验证步骤的类
class StepRunner:
    # 初始化方法，接受神经网络模型、损失函数、度量指标字典、优化器和学习率调度器等参数
    def __init__(self, net, loss_fn, metrics_dict=None, optimizer=None, lr_scheduler=None, stage="train"):
        self.net, self.loss_fn, self.metrics_dict, self.stage = net, loss_fn, metrics_dict, stage
        self.optimizer, self.lr_scheduler = optimizer, lr_scheduler

    def __call__(self, features, labels):
        # 前向传播，计算损失
        preds = self.net(features)
        loss = self.loss_fn(preds, labels)

        # 反向传播和优化（仅在训练阶段执行）
        if self.optimizer is not None and self.stage == "train":
            loss.backward()
            self.optimizer.step()
            if self.lr_scheduler is not None:
                self.lr_scheduler.step()
            self.optimizer.zero_grad()

        # 计算度量指标
        step_metrics = {self.stage + "_" + name: metric_fn(preds, labels).item()
                        for name, metric_fn in self.metrics_dict.items()}
        return loss.item(), step_metrics


# 定义一个用于执行一个训练或验证周期的类
class EpochRunner:
    # 初始化方法，接受StepRunner实例作为参数
    def __init__(self, steprunner):
        self.steprunner = steprunner
        self.stage = steprunner.stage
        self.steprunner.net.train() if self.stage == "train" else self.steprunner.net.eval()

    def __call__(self, dataloader):
        total_loss, step = 0, 0
        loop = tqdm(enumerate(dataloader), total=len(dataloader))
        for i, batch in loop:
            if self.stage == "train":
                loss, step_metrics = self.steprunner(*batch)
            else:
                with torch.no_grad():
                    loss, step_metrics = self.steprunner(*batch)
            step_log = dict({self.stage + "_loss": loss}, **step_metrics)

            total_loss += loss
            step += 1
            if i != len(dataloader) - 1:
                loop.set_postfix(**step_log)
            else:
                epoch_loss = total_loss / step
                epoch_metrics = {self.stage + "_" + name: metric_fn.compute().item()
                                 for name, metric_fn in self.steprunner.metrics_dict.items()}
                epoch_log = dict({self.stage + "_loss": epoch_loss}, **epoch_metrics)
                loop.set_postfix(**epoch_log)

                for name, metric_fn in self.steprunner.metrics_dict.items():
                    metric_fn.reset()
        return epoch_log


# 定义一个用于构建深度学习模型的类
class KerasModel(torch.nn.Module):
    # 初始化方法，接受神经网络模型、损失函数、度量指标字典、优化器和学习率调度器等参数
    def __init__(self, net, loss_fn, metrics_dict=None, optimizer=None, lr_scheduler=None):
        super().__init__()
        self.history = {}

        self.net = net
        self.loss_fn = loss_fn
        self.metrics_dict = nn.ModuleDict(metrics_dict)

        self.optimizer = optimizer if optimizer is not None else torch.optim.Adam(
            self.parameters(), lr=1e-2)
        self.lr_scheduler = lr_scheduler

    def forward(self, x):
        if self.net:
            return self.net.forward(x)
        else:
            raise NotImplementedError

    # 训练模型的方法
    def fit(self, train_data, val_data=None, epochs=10, ckpt_path='checkpoint.pt',
            patience=5, monitor="val_loss", mode="min"):

        for epoch in range(1, epochs + 1):
            printlog("Epoch {0} / {1}".format(epoch, epochs))

            # 1. 训练阶段
            train_step_runner = StepRunner(net=self.net, stage="train",
                                           loss_fn=self.loss_fn, metrics_dict=deepcopy(self.metrics_dict),
                                           optimizer=self.optimizer, lr_scheduler=self.lr_scheduler)
            train_epoch_runner = EpochRunner(train_step_runner)
            train_metrics = train_epoch_runner(train_data)

            for name, metric in train_metrics.items():
                self.history[name] = self.history.get(name, []) + [metric]

            # 2. 验证阶段
            if val_data:
                val_step_runner = StepRunner(net=self.net, stage="val",
                                             loss_fn=self.loss_fn, metrics_dict=deepcopy(self.metrics_dict))
                val_epoch_runner = EpochRunner(val_step_runner)
                with torch.no_grad():
                    val_metrics = val_epoch_runner(val_data)
                val_metrics["epoch"] = epoch
                for name, metric in val_metrics.items():
                    self.history[name] = self.history.get(name, []) + [metric]

            # 3. 提前停止
            if not val_data:
                continue
            arr_scores = self.history[monitor]
            best_score_idx = np.argmax(arr_scores) if mode == "max" else np.argmin(arr_scores)
            if best_score_idx == len(arr_scores) - 1:
                torch.save(self.net.state_dict(), ckpt_path)
                print("<<<<<< reach best {0} : {1} >>>>>>".format(monitor,
                                                                  arr_scores[best_score_idx]), file=sys.stderr)
            if len(arr_scores) - best_score_idx > patience:
                print("<<<<<< {} without improvement in {} epoch, early stopping >>>>>>".format(
                    monitor, patience), file=sys.stderr)
                break

        self.net.load_state_dict(torch.load(ckpt_path))
        return pd.DataFrame(self.history)

    # 评估模型性能的方法
    @torch.no_grad()
    def evaluate(self, val_data):
        val_step_runner = StepRunner(net=self.net, stage="val",
                                     loss_fn=self.loss_fn, metrics_dict=deepcopy(self.metrics_dict))
        val_epoch_runner = EpochRunner(val_step_runner)
        val_metrics = val_epoch_runner(val_data)
        return val_metrics

    # 使用模型进行预测的方法
    @torch.no_grad()
    def predict(self, dataloader):
        self.net.eval()
        result = torch.cat([self.forward(t[0]) for t in dataloader])
        return result.data


In [ ]:
from torchmetrics import Accuracy

# 创建一个新的神经网络模型net
net = Net()

# 使用KerasModel类构建一个模型对象model，传入以下参数：
# - net: 神经网络模型
# - loss_fn: 损失函数，这里使用二元交叉熵损失函数 (nn.BCEWithLogitsLoss())
# - optimizer: 优化器，这里使用Adam优化器 (torch.optim.Adam) 并传入网络参数和学习率
# - metrics_dict: 度量指标字典，包含一个二元分类准确率度量指标 (Accuracy)
model = KerasModel(net,
                   loss_fn=nn.BCEWithLogitsLoss(),
                   optimizer=torch.optim.Adam(net.parameters(), lr=0.01),
                   metrics_dict={"acc": Accuracy(task='binary')}
                   )


In [ ]:
# 使用模型对训练数据集 dl_train 进行训练
# - val_data: 验证数据集 dl_val，用于验证模型性能
# - epochs: 训练的总周期数
# - ckpt_path: 用于保存模型检查点的路径
# - patience: 提前停止的耐心，如果连续指定的度量没有改善，则提前停止训练
# - monitor: 监控的度量指标，在这里是验证集的准确率 'val_acc'
# - mode: 模式，'max' 表示监控的度量指标越大越好
model.fit(dl_train,
          val_data=dl_val,
          epochs=10,
          ckpt_path='checkpoint',
          patience=3,
          monitor='val_acc',
          mode='max')


### 四，评估模型

In [ ]:
# 导入 pandas 库
import pandas as pd

# 从模型对象 model 中获取训练过程中的历史记录
history = model.history

# 创建一个 DataFrame 对象 dfhistory，将历史记录存储在其中
dfhistory = pd.DataFrame(history)

# 打印 DataFrame，显示训练过程中的指标值和损失
dfhistory


In [ ]:
# 导入 matplotlib.pyplot 库，用于绘图
import matplotlib.pyplot as plt


# 定义一个函数 plot_metric，用于绘制训练和验证指标曲线
def plot_metric(dfhistory, metric):
    # 从历史记录 DataFrame 中获取训练和验证指标的数据
    train_metrics = dfhistory["train_" + metric]
    val_metrics = dfhistory['val_' + metric]

    # 获取训练周期数（x轴）
    epochs = range(1, len(train_metrics) + 1)

    # 绘制训练指标和验证指标的曲线
    plt.plot(epochs, train_metrics, 'bo--')  # 蓝色圆点线表示训练指标
    plt.plot(epochs, val_metrics, 'ro-')  # 红色实线表示验证指标

    # 设置图表标题、x轴标签、y轴标签和图例
    plt.title('Training and validation ' + metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_" + metric, 'val_' + metric])

    # 显示图表
    plt.show()


In [ ]:
plot_metric(dfhistory, "loss")

In [ ]:
plot_metric(dfhistory, "acc")

In [ ]:
# 评估
evaluation_result = model.evaluate(dl_val)
print(evaluation_result)


### 五，使用模型

In [ ]:
# 定义一个用于模型预测的函数 predict
def predict(net, dl):
    # 将模型设置为评估模式
    net.eval()
    with torch.no_grad():
        # 对数据加载器中的数据进行预测，并通过Sigmoid函数进行概率化
        result = nn.Sigmoid()(torch.cat([net.forward(t[0]) for t in dl]))
    return result.data


In [ ]:
y_pred_probs = predict(net, dl_val)
y_pred_probs

### 六，保存模型

In [ ]:
#模型权重已经被保存在了ckpt_path='checkpoint.'
net_clone = Net()
net_clone.load_state_dict(torch.load('checkpoint'))